In [1]:
import netCDF4
import itertools

from tqdm import tqdm

Nx = 512
Ny = 512
Nz = 1000
Na = 3
Ns = Nx * Ny

In [2]:
import itertools

from scipy import interpolate

import sys

def find_interpol_range(tau, Nz, Ns):

    thick = 1e+20
    thinn = 1e-20

    for k in range(Ns):

        opt_thick_end = tau[Nz - 1, k]
#       opt_thinn_end = tau[0, k]
        opt_thinn_end = tau[1, k]

        if opt_thinn_end > thinn: thinn = opt_thinn_end
        
        if opt_thick_end < thick: thick = opt_thick_end
    
    return np.log10(thinn), np.log10(thick)

def find_shortest_delta(tau, Nz, Ns):
    
    tau_1d = np.log10(tau.T).reshape(Nz * Ns)
    
    delta = np.abs(np.diff(tau_1d))
    
    idx_min = np.argmin(np.abs(delta - 0.001))
            
    return delta[idx_min]

def const_interpol_grid(tau, Nz, Ns):
    
    thin, thick = find_interpol_range(tau, Nz, Ns)
    delta =       find_shortest_delta(tau, Nz, Ns)
    
    Ng = int(np.floor(np.abs(thick - thin) / delta))
    
    grid = np.zeros(Ng)
    
    grid[0] = thin
    
    for i in range(Ng - 1): grid[i + 1] = grid[i] + delta
    
    return grid

def interpol_and_average(i, target, tau, T, igrid, Ns):
    
    T_mean = np.zeros(len(igrid))
    
    for k in tqdm(range(Ns), desc = 'interpolate and average ' + target):
    
        f = interpolate.interp1d(np.log10(tau[:, k]), T[:, k])
        
        Ti = f(igrid)
        
        T_mean += Ti
        
    T_mean /= Ns
    
    return T_mean

In [3]:
mu_id = ['10', '6', '2']

T_ssd =    np.ones((Na, Nz, Ns)) * np.nan
t200_ssd = np.ones((Na, Nz, Ns)) * np.nan

T_mag =    np.ones((Na, Nz, Ns)) * np.nan
t200_mag = np.ones((Na, Nz, Ns)) * np.nan

grid_ssd = np.ones((Na, 110000)) * np.nan
grid_mag = np.ones((Na, 110000)) * np.nan

Ti_ssd =   np.ones((Na, 110000)) * np.nan
Ti_mag =   np.ones((Na, 110000)) * np.nan

for i in range(Na):
    
    print("---------------")
    print('mu = ', mu_id[i])
    print("---------------")
    
    if i == 0:           mu_mod = mu_id[i]
    
    if i == 1 or i == 2: mu_mod = '0' + mu_id[i]    
    
    print('SSD cube')
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    print('reading tau...')
    
    t200 = np.array(netCDF4.Dataset('./rot_tau200/222809_mu_' + mu_mod + '/result_ttau200.222809.nc')['tau'])
    
    Nz_cur = len(t200[:, 0, 0])
    
    t200_ssd[i, : Nz_cur, :] = t200.reshape(Nz_cur, Ny * Nx)
    
    if i == 1 or i == 2: t200_ssd[i, 0, :] = 1e-15
    
    print('sampling tau...')
    tau = t200_ssd[i, : Nz_cur, sample].T
    
    grid = const_interpol_grid(tau, Nz_cur, Ns)
    
    grid_ssd[i, : len(grid)] = 10.0**grid
    
    print('reading temperature...')
    
    T = np.array(netCDF4.Dataset('./rot_tau200/222809_mu_' + mu_mod + '/T_mu_' + mu_id[i] + '.222809.nc')['T'])
    
    T_ssd[i, : Nz_cur, :] = T.reshape(Nz_cur, Ny * Nx)
    
    T_mean = interpol_and_average(i, 'SSD temp', tau, T_ssd[i, : Nz_cur, sample].T, grid, Ns)

    Ti_ssd[i, : len(T_mean)] = T_mean
    
    print('300G cube')
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    print('reading tau...')
    
    t200 = np.array(netCDF4.Dataset('./rot_tau200/627321_mu_' + mu_mod + '/result_ttau200.627321.nc')['tau'])
    
    Nz_cur = len(t200[:, 0, 0])
    
    t200_mag[i, : Nz_cur, :] = t200.reshape(Nz_cur, Ny * Nx)
    
    if i == 1 or i == 2: t200_mag[i, 0, :] = 1e-15
    
    print('sampling tau...')
    tau = t200_mag[i, : Nz_cur, sample].T
    
    grid = const_interpol_grid(tau, Nz_cur, Ns)
    
    grid_mag[i, : len(grid)] = 10.0**grid
    
    print('reading temperature...')
    
    T = np.array(netCDF4.Dataset('./rot_tau200/627321_mu_' + mu_mod + '/T_mu_' + mu_id[i] + '.627321.nc')['T'])
    
    T_mag[i, : Nz_cur, :] = T.reshape(Nz_cur, Ny * Nx)
    
    T_mean = interpol_and_average(i, '300G temp', tau, T_mag[i, : Nz_cur, sample].T, grid, Ns)

    Ti_mag[i, : len(T_mean)] = T_mean
    
np.savez('tau200_rot_interpol', grid_ssd = grid_ssd, Ti_ssd = Ti_ssd,\
                                grid_mag = grid_mag, Ti_mag = Ti_mag)

---------------
mu =  10
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...


interpolate and average SSD temp: 100%|██████████| 262144/262144 [01:02<00:00, 4187.23it/s]


300G cube
reading tau...
sampling tau...
reading temperature...


interpolate and average 300G temp: 100%|██████████| 262144/262144 [00:58<00:00, 4479.99it/s]


---------------
mu =  6
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...


...

---------------
mu =  2
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...


interpolate and average SSD temp: 100%|██████████| 262144/262144 [02:20<00:00, 1872.19it/s]


300G cube
reading tau...
sampling tau...
reading temperature...


interpolate and average 300G temp: 100%|██████████| 262144/262144 [03:33<00:00, 1229.45it/s]


NameError: name 'grid_300' is not defined

In [1]:
import netCDF4

from tqdm import tqdm

Na = 3
Nz = 1000
Nx = 512
Ny = 512

T_ssd =      np.ones((Na, Nz, Ny * Nx)) * np.nan
tau200_ssd = np.ones((Na, Nz, Ny * Nx)) * np.nan

T_mag =      np.ones((Na, Nz, Ny * Nx)) * np.nan
tau200_mag = np.ones((Na, Nz, Ny * Nx)) * np.nan

mu = ['10', '6', '2']

for i in range(Na):
    
    if i == 0:           mu_mod = mu[i]
    
    if i == 1 or i == 2: mu_mod = '0' + mu[i]
        
    print('Reading angle ' + mu_mod + '...')

    T =      np.array(netCDF4.Dataset('./rot_tau200/222809_mu_' + mu_mod + '/T_mu_' + mu[i] + '.222809.nc')['T'])
    
    Nz_cur = len(T[:, 0, 0])
    
    T_ssd[i, : Nz_cur, :] = T.reshape(Nz_cur, Ny * Nx)
    
    T =      np.array(netCDF4.Dataset('./rot_tau200/627321_mu_' + mu_mod + '/T_mu_' + mu[i] + '.627321.nc')['T'])
    
    T_mag[i, : Nz_cur, :] = T.reshape(Nz_cur, Ny * Nx)

    tau200 = np.array(netCDF4.Dataset('./rot_tau200/222809_mu_' + mu_mod + '/result_ttau200.222809.nc')['tau'])
    
    tau200_ssd[i, : Nz_cur, :] = tau200.reshape(Nz_cur, Ny * Nx)

    tau200 = np.array(netCDF4.Dataset('./rot_tau200/627321_mu_' + mu_mod + '/result_ttau200.627321.nc')['tau'])
    
    tau200_mag[i, : Nz_cur, :] = tau200.reshape(Nz_cur, Ny * Nx)

Reading angle 10...
Reading angle 06...
Reading angle 02...


In [6]:
tau200 = np.array(netCDF4.Dataset('./rot_tau200/627321_mu_10/result_ttau200.627321.nc')['tau'])

In [ ]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.8', '0.6', '0.4', '0.2']

idx_mu = [0, 2, 4]

fig = plt.figure(dpi = 300)

fig.set_size_inches(5, 11.25 * 2 / 5, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(2, 2, width_ratios = [1, 1], hspace = 0.2, wspace = 0.3)

tem_ssd = plt.subplot(gs[0, 0])
t_ssd_c = plt.subplot(gs[0, 1])

tem_300 = plt.subplot(gs[1, 0])
t_300_c = plt.subplot(gs[1, 1])

for i in range(len(idx_mu)):

    tem_ssd.plot(grid_ssd[idx_mu[i], :], Ti_ssd[idx_mu[i], :], label = r'$\mu = $' + mus[idx_mu[i]])
    t_ssd_c.plot(grid_ssd[idx_mu[i], :], Ti_ssd[idx_mu[i], :])
    
    tem_300.plot(grid_300[idx_mu[i], :], Ti_300[idx_mu[i], :])
    t_300_c.plot(grid_300[idx_mu[i], :], Ti_300[idx_mu[i], :])
    
    tem2_ssd = t_ssd_c.twinx()
    tem2_300 = t_300_c.twinx()
    
    tem2_ssd.yaxis.set_ticks([])
    tem2_300.yaxis.set_ticks([])

    tem_ssd.set_xscale('log')
    t_ssd_c.set_xscale('log')
    tem_300.set_xscale('log')
    t_300_c.set_xscale('log')

    tem_ssd.set_xlim(3e+1, 2e-10)
    tem_300.set_xlim(tem_ssd.get_xlim())

    t_ssd_c.set_xlim(1e-3, 2e-10)
    t_300_c.set_xlim(t_ssd_c.get_xlim())

    tem_ssd.set_ylim(3000, 10000)
    tem_300.set_ylim(3000, 10000)
    
    t_ssd_c.set_ylim(3000, 4500)
    t_300_c.set_ylim(3400, 4500)

    tem_ssd.yaxis.set_minor_locator(AutoMinorLocator(4))
    tem_300.yaxis.set_minor_locator(AutoMinorLocator(4))

    tem_ssd.set_ylabel('Temperature, K')
    tem_300.set_ylabel('Temperature, K')
    
    tem2_ssd.set_ylabel('SSD')
    tem2_300.set_ylabel('300G')
    
    tem_300.set_xlabel(r'$\tau_\mathrm{Ross}$')
    t_300_c.set_xlabel(r'$\tau_\mathrm{Ross}$')
    
    tem_ssd.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('intro_img_2_tpd_cut.pdf', bbox_inches = 'tight')

plt.close('all')